In [7]:
# 임포트 및 GPU사용, colab에 mount, 시드 부여 등 학습 모델 외적으로 사용하는 코드들
import torch
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader
import random
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

from google.colab import drive
drive.mount('/content/drive')

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

cuda
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
trans=transforms.Compose([transforms.Resize((320,256)),transforms.ToTensor()])
train_data = dsets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/traing_dataset', transform=trans)
data_loader = DataLoader(dataset = train_data, batch_size = 40,shuffle = True, num_workers=2)

In [9]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 Input shape=(?, 3, 320, 256)
        #    Conv     -> (?, 32, 320, 256)
        #    Pool     -> (?, 32, 160, 128)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 Input shape=(?, 32, 160, 128)
        #    Conv      ->(?, 64, 160, 128)
        #    Pool      ->(?, 64, 80, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 Input shape=(?, 64, 80, 64)
        #    Conv      ->(?, 128, 80, 64)
        #    Pool      ->(?, 128, 40, 32)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L4 Linear 128x40x32 inputs -> 625 outputs
        self.layer4 = torch.nn.Sequential(
            torch.nn.Linear(128*40*32, 625, bias=True),
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.5))  #Dropout이란 뉴럴네트워크의 경로 몇개를 버리면서 학습하는 것으로, overfitting을 방지하는 효과를 가진다.
        # L5 Linear 625 inputs -> 3 outputs
        self.layer5 = torch.nn.Linear(625, 3, bias=True)
        # 자비에르 유니폼: Linear의 초깃값을 0이아닌 자비에르방식으로 설정.
        torch.nn.init.xavier_uniform_(self.layer5.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC , Linear연산을 하기 전에 한줄 벡터로 변환하여 입력한다.
        out = self.layer4(out)
        out = self.layer5(out)
        return out

In [10]:
# parameters
learning_rate = 0.0005
training_epochs = 10

In [11]:
# contruct model
model = CNN().to(device)

# define cost/loss & optimizer, Multinominal 알고리즘을 사용하여 CrossEntropyLoss function을 사용하였고, SGD모델과 유사한 Adam모델을 옵티마이저로 사용하였다.
criterion = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
##### 학습이 이루어지는 loop
##### 지난 source code를 그대로 사용해도 되지만, 연습삼아 직접 작성해보자
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for num,data in enumerate(data_loader):
        X , Y = data 
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.403838307
[Epoch:    2] cost = 0.107941732
[Epoch:    3] cost = 0.0539666675
[Epoch:    4] cost = 0.0425397456
[Epoch:    5] cost = 0.0365260132
[Epoch:    6] cost = 0.0243192725
[Epoch:    7] cost = 0.0211366024
[Epoch:    8] cost = 0.0173476059
[Epoch:    9] cost = 0.00908318534
[Epoch:   10] cost = 0.0043152771
Learning Finished!


In [14]:
trans=transforms.Compose([transforms.Resize((320,256)),transforms.ToTensor()])
test_data = dsets.ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/test_dataset', transform=trans)
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

with torch.no_grad():
    for num,data in enumerate(data_loader):
        X , Y = data 
        X = X.to(device)
        Y = Y.to(device)

    prediction = model(X)
    correct_prediction = torch.argmax(prediction, 1) == Y
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f717b132050><function _MultiProcessingDataLoaderIter.__del__ at 0x7f717b132050>

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
      File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
self._shutdown_workers()    
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
self._shutdown_workers()    
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
if w.is_alive():    
if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child

Accuracy: 1.0
